# Deploy YANG Data Model to Stream CPU Utilization Data Using Model-driven Telemetry
##### <sup>Authored by Veena Manuel (veedas@cisco.com)</sup>

The use case illustrates how, with the YANG data model, you can stream telemetry data about CPU utilization. Monitoring CPU utilization ensures efficient storage capabilities in your network. In a Dial-out mode, the router dials out to the receiver to establish a subscription-based telemetry session.

The following image represents a topology with 4 routers; router P1 is connected to a collector to stream telemetry data.

![Topology](Telemetry-topology.png)

Telemetry involves the following workflow:
- Define: You define a subscription to stream data from the router to the receiver. To define a subscription,
you create a destination-group and a sensor-group.
- Deploy: The router establishes a subscription-based telemetry session and streams data to the receiver.
You verify subscription deployment on the router.
- Operate: You consume and analyse telemetry data using open-source tools, and take necessary actions
based on the analysis.

To gain an indepth understanding about telemetry streaming modes, see the [Telemetry Configuration Guide](https://www.cisco.com/c/en/us/support/routers/8000-series-routers/products-installation-and-configuration-guides-list.html).

## Configuration steps
* [Bring up emulator session with the base network and configurations](#step1)
* [Configure a Telemetry Session Between the Router and the Collector](#step2)
* [Verify the Telemetry configuration](#step3)
* [Operate on Telemetry Data](#step4)

## <a name="step1"></a>Bring up emulator session with the base network and configurations

In order to set up the emulator, you have to execute the cell below by clicking the play button at the top of this notebook. 

In the cell below, the python module **telemetry** sets up the python enviroment and the simulated router test bed along with the base configurations of IPv4 addresses, OSPF configurations, MPLS configurations, Loopback interfaces, etc. After the import, the below code block initialises and brings up the emulator using **sim.start** API call.

>The bring up can be slow, 10 minutes +.


In [ ]:
from telemetry import *
sim = Vxr()
sim.no_image_copy=True

sim.clean()
print("Sim clean: Done")
print("Simulation starting. Please wait for the Sim status message. This may take 3-10 minutes.")

try:
    sim.start(cfg)
    status = sim.status()
    print("Sim status: ", status)
except Exception as err:
    print("Sim launch failed (%s)" % str(err))

>At this point, console access to the router is available. Optionally, you can access the simulated router consoles directly from your laptop  through ssh.

In [2]:
print('Consoles can be reached by:')
print('P1:', get_telnet_cmd(sim, 'rp1'), '\nP3:', get_telnet_cmd(sim, 'rp3'), '\nPE1:', get_telnet_cmd(sim, 'rpe1'), '\nPE3:', get_telnet_cmd(sim, 'rpe3'), '\nser1:', get_telnet_cmd(sim, 'ser1'))
print('or better:')
print('P1:', get_ssh_cmd(sim, 'rp1'), '\nP3:', get_ssh_cmd(sim, 'rp3'), '\nPE1:', get_ssh_cmd(sim, 'rpe1'),  '\nPE3:', get_ssh_cmd(sim, 'rpe3'), '\nser1:', get_ssh_cmd(sim, 'ser1')),
print('The password is cisco123')

Consoles can be reached by:
P1: telnet 172.17.0.2 41145 
P3: telnet 172.17.0.2 40160 
PE1: telnet 172.17.0.2 33849 
PE3: telnet 172.17.0.2 39988 
ser1: telnet 172.17.0.2 36292
or better:
P1: ssh cisco@172.17.0.2 -p64966 
P3: ssh cisco@172.17.0.2 -p60916 
PE1: ssh cisco@172.17.0.2 -p60537 
PE3: ssh cisco@172.17.0.2 -p62351 
ser1: ssh cisco@172.17.0.2 -p60508
The password is cisco123


In [3]:
ports = sim.ports()
ser1_ipaddress = str(ports['ser1']['HostAgent'])
ser1_sshport = str(ports['ser1']['xr_redir22'])
p1_ipaddress = str(ports['rp1']['HostAgent'])
p1_sshport = str(ports['rp1']['xr_redir22'])

Install ncclient to talk to NETCONF-enabled devices. Here, router P1.

In [4]:
%%capture coutput
!pip install ncclient

## <a name="step3"></a>Configure a telemetry session between the router and the collector

To configure telemetry on the router, you will need to setup the following values for a dial-out mode:
- Create a destination group: Create one or more destinations to collect telemetry data from a router. Define a destination-group to contain the details about the destinations. Include the destination address (ipv4 or ipv6), port, transport, and encoding format in the destination-group.
> This use case uses 'self-describing-gpb' encoding and 'UDP' as the transport protocol. If you use TCP or gRPC, ensure that a receiver is setup at the collector. The receiver must match the values for encoding, transport and port that you set in the telemetry configuration.

- Create a Sensor-group: Specify the subset of the data that you want to stream from the router using sensor paths. The Sensor path represents the path in the hierarchy of a YANG data model. Create a sensor-group to contain the sensor paths.
> Here, the sensor path is for streaming CPU utilization.

- Create a Subscription: Subscribe to telemetry data that is streamed from a router. A Subscription binds the destination-group with the sensor-group and sets the streaming method. The streaming method can be Cadence-driven or Event-driven telemetry. Cadence-driven telemetry continually streams data (operational statistics and state transitions) at a configured cadence. Whereas, event-driven telemetry optimizes data that is collected at the receiver and streams data only when a state transition occurs.

>Port number range <1-65535>.

> IMPORTANT: Do not forget to replace the destination address and port number

In [5]:
from ncclient import manager
from ncclient.xml_ import to_ele
from lxml import etree

telemetry = '''
<config>
<telemetry-model-driven xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-telemetry-model-driven-cfg">
   <destination-groups>
    <destination-group>
     <destination-id>D1</destination-id>
     <ipv4-destinations>
      <ipv4-destination>
       <ipv4-address>192.168.122.20</ipv4-address>
       <destination-port>20030</destination-port>
       <encoding>self-describing-gpb</encoding>
       <protocol>
        <protocol>udp</protocol>
       </protocol>
      </ipv4-destination>
     </ipv4-destinations>
    </destination-group>
   </destination-groups>
   <sensor-groups>
    <sensor-group>
     <sensor-group-identifier>SGroup1</sensor-group-identifier>
     <sensor-paths>
      <sensor-path>
       <telemetry-sensor-path>Cisco-IOS-XR-wdsysmon-fd-oper:system-monitoring/cpu-utilization</telemetry-sensor-path>
      </sensor-path>
     </sensor-paths>
    </sensor-group>
   </sensor-groups>
   <enable></enable>
   <subscriptions>
    <subscription>
     <subscription-identifier>Sub1</subscription-identifier>
     <sensor-profiles>
      <sensor-profile>
       <sensorgroupid>SGroup1</sensorgroupid>
       <sample-interval>3000</sample-interval>
      </sensor-profile>
     </sensor-profiles>
     <destination-profiles>
      <destination-profile>
       <destination-id>D1</destination-id>
      </destination-profile>
     </destination-profiles>
    </subscription>
   </subscriptions>
  </telemetry-model-driven>
 </config>
 '''

def connect(host, port, user, password, source):
      conn = manager.connect(host=host,
                             port=port,
                             username=user,
                             password = password,
                             device_params={'name': 'iosxr'},
                             hostkey_verify=False,
                             allow_agent=False,
                             look_for_keys=False
                            )

      rpc_reply = conn.edit_config(config=telemetry)
      conn.commit()
      print(rpc_reply)

connect(p1_ipaddress, p1_sshport, 'cisco', 'cisco123', 'candidate')

<?xml version="1.0"?>
<rpc-reply message-id="urn:uuid:4cc55f5f-d7c7-43b3-ad45-3e291cd4760a" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0">
 <ok/>
</rpc-reply>



## <a name="step4"></a>Verify the telemetry configuration

Access the telnet consoles of the router to apply and verify the telemetry configuration.

In [6]:
console_ports = sim.ports()
loginp1 = telnetlib.Telnet(str(console_ports['rp1']['HostAgent']) , str(console_ports['rp1']['serial0']))

Verify the deployment of the subscription on the router.

In [7]:
# Show the telemetry configuration applied on the router.
loginp1.write(('''
show running-config telemetry model-driven
''').encode('ascii'))
line = loginp1.read_until(b'/r/n',2)
print("***** VIEW FROM TELNET CONSOLE OF P1 *****")
print(line.decode())

***** VIEW FROM TELNET CONSOLE OF P1 *****

RP/0/RP0/CPU0:P1#show running-config telemetry model-driven
Wed Apr 21 12:21:32.957 UTC
telemetry model-driven
 destination-group D1
  address-family ipv4 192.168.122.20 port 20030
   encoding self-describing-gpb
   protocol udp
  !
 !
 sensor-group SGroup1
  sensor-path Cisco-IOS-XR-wdsysmon-fd-oper:system-monitoring/cpu-utilization
 !
 subscription Sub1
  sensor-group-id SGroup1 sample-interval 3000
  destination-id D1
 !
!

RP/0/RP0/CPU0:P1#


Verify the telemetry configuration on the router.

In [8]:
# Show the telemetry configuration applied on the router.
loginp1.write(('''
show telemetry model-driven subscription Sub1
''').encode('ascii'))
line = loginp1.read_until(b'/r/n',2)
print("***** VIEW FROM TELNET CONSOLE OF P1 *****")
print(line.decode())

***** VIEW FROM TELNET CONSOLE OF P1 *****

RP/0/RP0/CPU0:P1#show telemetry model-driven subscription Sub1
Wed Apr 21 12:21:36.406 UTC
Subscription:  Sub1
-------------
  State:       NA
  Sensor groups:
  Id: SGroup1
    Sample Interval:      3000 ms
    Heartbeat Interval:   NA
    Sensor Path:          Cisco-IOS-XR-wdsysmon-fd-oper:system-monitoring/cpu-utilization
    Sensor Path State:    Not Resolved

  Destination Groups:
  Group Id: D1
    Destination IP:       192.168.122.20
    Destination Port:     20030
    Encoding:             self-describing-gpb
    Transport:            udp
    State:                NA
    TLS :                 False

  Collection Groups:
  ------------------
  No active collection groups

RP/0/RP0/CPU0:P1#


>After the telemetry session is established, the router streams data to the receiver to create a data lake. Check the collector console to see the streamed data.

## <a name="step5"></a>Operate on telemetry data

You can start consuming and analyzing telemetry data from the data lake using an open-sourced collection
stack based on your requirement. 
For example, the following tools can be used to analyze the collected telemetry data:
- Pipeline is a lightweight tool used to collect data. You can download Network Telemetry Pipeline from Github. You define how you want the collector to interact with routers and where you want to send the
processed data using pipeline.conf file.
- Telegraph (plugin-driven server agent) and InfluxDB (a time series database (TSDB)) stores telemetry
data, which is retrieved by visualization tools. You can download InfluxDB from Github. You define
what data you want to include into your TSDB using the metrics.json file.
- Grafana is a visualization tool that displays graphs and counters for data streamed from the router.

For more information, see http://xrdocs.io/telemetry/.

#### Clean-up the topology

>Once you are done with experimenting on the topology, you should bring down the emulator by executing the following steps:

In [9]:
# close our telnet
loginp1.close()

In [10]:
sim.stop()

In [11]:
sim.clean()

In [ ]:
# Clean up sim scratch space
shutil.rmtree(sim_dir)